### Requires
1. Sampling Rate : 16000 
2. Channels : Mono
3. Type : Wav
4. Not more that 1 min (5 second to be safe)

In [1]:
import subprocess
subprocess.call('rm temp.wav',shell=True)
#converting mp3 to wav
subprocess.call('ffmpeg -i {} -acodec pcm_s16le -ar 16000 -ac 1 temp.wav'.format('dialog1.mp3'),shell=True)
#removing previous present files
subprocess.call('rm parts/*', shell=True)
#segmenting our .wav in parts of 5 seconds for processing 
subprocess.call('ffmpeg -i temp.wav -f segment -segment_time 5 -c copy parts/out%09d.wav',shell=True)

1

In [2]:
import os
# API KEY : Issued in the Name of Aditya Jain : Please do not share
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="../Audio-to-speech-aa00874e061f.json"
files = os.listdir('parts')
sorted(files)

['out000000000.wav',
 'out000000001.wav',
 'out000000002.wav',
 'out000000003.wav',
 'out000000004.wav',
 'out000000005.wav',
 'out000000006.wav',
 'out000000007.wav',
 'out000000008.wav',
 'out000000009.wav',
 'out000000010.wav',
 'out000000011.wav',
 'out000000012.wav']

In [3]:
from google.cloud import speech_v1p1beta1 as speech
from tqdm import tqdm
client = speech.SpeechClient()

conversation = list()

for file in tqdm(sorted(files)):

    with open('parts/'+file, 'rb') as audio_file:
        content = audio_file.read()

    audio = speech.types.RecognitionAudio(content=content)

    config = speech.types.RecognitionConfig(
        encoding=speech.enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code='en-US',
        enable_speaker_diarization=True,
        diarization_speaker_count=2)

    response = client.recognize(config, audio)

    # The transcript within each result is separate and sequential per result.
    # However, the words list within an alternative includes all the words
    # from all the results thus far. Thus, to get all the words with speaker
    # tags, you only have to take the words list from the last result:
    result = response.results[-1]

    words_info = result.alternatives[0].words
    current = words_info[0].speaker_tag
    # Printing out the output:
    ongoing = {words_info[0].speaker_tag:[]}
    for word_info in words_info:
        #print("word: '{}', speaker_tag: {}".format(word_info.word,word_info.speaker_tag))
        if word_info.speaker_tag != current:
            conversation.append(ongoing)
            ongoing = { word_info.speaker_tag :[] }
            current = word_info.speaker_tag
            ongoing[word_info.speaker_tag].append(word_info.word)
        else:
            ongoing[word_info.speaker_tag].append(word_info.word)
    conversation.append(ongoing)

DefaultCredentialsError: File ../Audio-to-speech-aa00874e061f.json was not found.

In [ ]:
for converse in conversation:
    print( "Speaker {} : {}".format( list(converse.keys())[0], " ".join(converse[list(converse.keys())[0]]) ) )

Speaker 1 : hi I'm homeless what's your name
Speaker 2 : my name is Ricky
Speaker 1 : ECU new student here
Speaker 2 : I have my first lesson this morning
Speaker 1 : you and you student to
Speaker 2 : know I've been here for six months
Speaker 2 : a long time so long really
Speaker 1 : 3 what about you I'm in Xbox one who's your teacher
Speaker 1 : don't remember
Speaker 2 : her name but she's got curly red hair
Speaker 1 : yes I think so that's probably an Wallace
Speaker 2 : do you know her is she your teacher to know but she told me last time
Speaker 2 : have you been here only a week. Long would you live
Speaker 2 : well I'm staying at the YMCA at the moment I'm looking for someone more permanent
Speaker 1 : do you know of any good places
Speaker 2 : actually my friend has a spare room
Speaker 2 : are apartment and she's looking for a flatmate phone number
Speaker 1 : thanks for your help can I buy you a coffee


### Advantages
1. diarization available (seperating speakers)
2. Very fast
3. Easy to use, Great documentation

### Disadvantages
1. Difficult to process nouns like names, places etc
2. Omits some part of speech sometimes, but most words are recognized.
3. Paid on basis of monthly amount of characters processed
4. Uses internet connection, since it is a API

### Other Options
1. PocketSphinx - works offline, lack of documentation, Mainly for Java
2. IBM speech to text - online, paid, not good as google cloud speech to text
3. Microsoft Bing Voice Recognition - online, paid, not good as google cloud speech to text
4. Many more which are not famous.